In [ ]:
from datasets import load_dataset
base_url = "https://storage.googleapis.com/huggingface-nlp/cache/datasets/wikipedia/20200501.en/1.0.0/"
data_files = {"train": base_url + "wikipedia-train.parquet"}
wiki = load_dataset("parquet", data_files=data_files, split="train", streaming=True)
print(next(iter(wiki)))
# {'title': 'Yangliuqing', 'text': 'Yangliuqing () is a market town in Xiqing District...'}